NAMA : Mohammad Nurdin Prastya Hermansah

Prodi : D4 Teknik Elektronika

NIM: 20507334047

# Machine Learning Exercise 3 - Multi-Class Classification

For this exercise we'll use logistic regression to recognize hand-written digits (0 to 9).  We'll be extending the implementation of logistic regression we wrote in exercise 2 and apply it to one-vs-all classification.  Let's get started by loading the data set.  It's in MATLAB's native format, so to load it in Python we need to use a SciPy utility.

**Artinya**

Pada latihan ini kita akan menggunakan logistic regression to mengenali tulisan tangan digits(0-9). Kita akan memperluas penerapan regresi logistik yang kita tulis di latihan 2 dan menerapkannya pada klasifikasi one-vs-all. Mari kita mulai dengan memuat kumpulan data. Itu dalam format asli MATLAB, jadi untuk memuatnya dengan Python kita perlu menggunakan utilitas SciPy.

In [1]:
# import library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [ ]:
# import data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# membaca data dan menampilkanya
data = loadmat('/content/drive/MyDrive/PRAKTEK/Data Praktek/ex3data.mat')
data['X'].shape, data['y'].shape

((5000, 400), (5000, 1))

Great, we've got our data loaded.  The images are represented in matrix X as a 400-dimensional vector (of which there are 5,000 of them).  The 400 "features" are grayscale intensities of each pixel in the original 20 x 20 image.  The class labels are in the vector y as a numeric class representing the digit that's in the image.

The first task is to modify our logistic regression implementation to be completely vectorized (i.e. no "for" loops).  This is because vectorized code, in addition to being short and concise, is able to take advantage of linear algebra optimizations and is typically much faster than iterative code.  However if you look at our cost function implementation from exercise 2, it's already vectorized!  So we can re-use the same implementation here.  Note we're skipping straight to the final, regularized version.

**Artinya**

Bagus, data kita sudah dimuat. Gambar direpresentasikan dalam matriks X sebagai vektor 400 dimensi (yang jumlahnya 5.000). 400 "fitur" adalah intensitas skala abu-abu dari setiap piksel dalam gambar asli 20 x 20. Label kelas ada di vektor y sebagai kelas numerik yang mewakili digit yang ada di gambar.

Tugas pertama adalah memodifikasi implementasi regresi logistik kita menjadi sepenuhnya vektor (i.e. no "for" loops). Ini karena kode vektor, selain pendek dan ringkas, mampu memanfaatkan pengoptimalan aljabar linier dan biasanya jauh lebih cepat daripada kode iteratif. Namun jika Anda melihat implementasi fungsi cost kita dari latihan 2, itu sudah di-vektor-kan! Jadi kita dapat menggunakan kembali implementasi yang sama di sini. Perhatikan bahwa kami langsung melompat ke versi final yang telah diatur.

Langkah pertama kita perlu membuat fungsi sigmoid. Kode untuk ini cukup sederhana.

In [5]:
# function sigmoid
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

sekarang kita harus menulis fungsi cost untuk mengevaluasi solusi.

In [6]:
# evaluate solution
def cost(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
    reg = (learningRate / 2 * len(X)) * np.sum(np.power(theta[:,1:theta.shape[1]], 2))
    return np.sum(first - second) / (len(X)) + reg

Next we need the function that computes the gradient.  Again, we already defined this in the previous exercise, only in this case we do have a "for" loop in the update step that we need to get rid of.  Here's the original code for reference:

**Artinya**

Selanjutnya kita membutuhkan fungsi yang menghitung gradien. Sekali lagi, kita telah mendefinisikan ini pada latihan sebelumnya, hanya dalam kasus ini kita memiliki loop "untuk" pada langkah pembaruan yang perlu kita singkirkan. Berikut kode asli untuk referensi:

In [7]:
# computing gradient with "for" loop in the update step that we need to get rid of.
def gradient_with_loop(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    parameters = int(theta.ravel().shape[1])
    grad = np.zeros(parameters)
    
    error = sigmoid(X * theta.T) - y
    
    for i in range(parameters):
        term = np.multiply(error, X[:,i])
        
        if (i == 0):
            grad[i] = np.sum(term) / len(X)
        else:
            grad[i] = (np.sum(term) / len(X)) + ((learningRate / len(X)) * theta[:,i])
    
    return grad

In our new version we're going to pull out the "for" loop and compute the gradient for each parameter at once using linear algebra (except for the intercept parameter, which is not regularized so it's computed separately).  To follow the math behind the transformation, refer to the exercise 3 text.

Also note that we're converting the data structures to NumPy matrices. This is done in an attempt to make the code look more similar to Octave than it would using arrays because matrices automatically follow matrix operation rules vs. element-wise operations, which is the default for arrays.  There is some debate in the community over wether or not the matrix class should be used at all, but it's there so we're using it in these examples.

**Artinya**

Dalam versi baru kami, kami akan mengeluarkan loop "untuk" dan menghitung gradien untuk setiap parameter sekaligus menggunakan aljabar linier (kecuali untuk parameter intersep, yang tidak diatur sehingga dihitung secara terpisah). Untuk mengikuti matematika di balik transformasi, lihat teks latihan 3.

Perhatikan juga bahwa kami mengonversi struktur data menjadi matriks NumPy. Ini dilakukan dalam upaya untuk membuat kode terlihat lebih mirip dengan Oktaf daripada menggunakan array karena matriks secara otomatis mengikuti aturan operasi matriks vs. operasi berdasarkan elemen, yang merupakan default untuk array. Ada beberapa perdebatan di komunitas tentang apakah kelas matriks harus digunakan atau tidak, tetapi itu ada di sana, jadi kami menggunakannya dalam contoh ini.


In [8]:
#  menghitung gradien 
def gradient(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    parameters = int(theta.ravel().shape[1])
    error = sigmoid(X * theta.T) - y
    
    grad = ((X.T * error) / len(X)).T + ((learningRate / len(X)) * theta)
    
    # intercept gradient is not regularized
    grad[0, 0] = np.sum(np.multiply(error, X[:,0])) / len(X)
    
    return np.array(grad).ravel()

Now that we've defined our cost and gradient functions, it's time to build a classifier.  For this task we've got 10 possible classes, and since logistic regression is only able to distiguish between 2 classes at a time, we need a strategy to deal with the multi-class scenario.  In this exercise we're tasked with implementing a one-vs-all classification approach, where a label with k different classes results in k classifiers, each one deciding between "class i" and "not class i".  We're going to wrap the classifier training up in one function that computes the final weights for each of the 10 classifiers and returns the weights as a k X (n + 1) array, where n is the number of parameters.

**Artinya**

Sekarang setelah kita menentukan fungsi cost dan gradien, saatnya membuat classifier. Untuk tugas ini, kami memiliki 10 kemungkinan kelas, dan karena regresi logistik hanya dapat membedakan antara 2 kelas sekaligus, kami memerlukan strategi untuk menghadapi skenario multi-kelas. Dalam latihan ini kita ditugaskan untuk mengimplementasikan pendekatan klasifikasi satu lawan semua, di mana sebuah label dengan k kelas yang berbeda menghasilkan k pengklasifikasi, masing-masing memutuskan antara "kelas i" dan "bukan kelas i". Kita akan membungkus pelatihan pengklasifikasi dalam satu fungsi yang menghitung bobot akhir untuk masing-masing dari 10 pengklasifikasi dan mengembalikan bobot sebagai larik k X (n + 1), di mana n adalah jumlah parameter.


In [9]:
# import library scipy
from scipy.optimize import minimize

# membuat classifier
def one_vs_all(X, y, num_labels, learning_rate):
    rows = X.shape[0]
    params = X.shape[1]
    
    # k X (n + 1) array for the parameters of each of the k classifiers
    all_theta = np.zeros((num_labels, params + 1))
    
    # insert a column of ones at the beginning for the intercept term
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    
    # labels are 1-indexed instead of 0-indexed
    for i in range(1, num_labels + 1):
        theta = np.zeros(params + 1)
        y_i = np.array([1 if label == i else 0 for label in y])
        y_i = np.reshape(y_i, (rows, 1))
        
        # minimize the objective function
        fmin = minimize(fun=cost, x0=theta, args=(X, y_i, learning_rate), method='TNC', jac=gradient)
        all_theta[i-1,:] = fmin.x
    
    return all_theta

A few things to note here...first, we're adding an extra parameter to theta (along with a column of ones to the training data) to account for the intercept term.  Second, we're transforming y from a class label to a binary value for each classifier (either is class i or is not class i).  Finally, we're using SciPy's newer optimization API to minimize the cost function for each classifier.  The API takes an objective function, an initial set of parameters, an optimization method, and a jacobian (gradient) function if specified.  The parameters found by the optimization routine are then assigned to the parameter array.

One of the more challenging parts of implementing vectorized code is getting all of the matrix interactions written correctly, so I find it useful to do some sanity checks by looking at the shapes of the arrays/matrices I'm working with and convincing myself that they're sensible.  Let's look at some of the data structures used in the above function.

**Artinya**

Beberapa hal yang perlu diperhatikan di sini ... pertama, kami menambahkan parameter tambahan ke theta (bersama dengan kolom satu ke data pelatihan) untuk memperhitungkan istilah intersep. Kedua, kami mengubah y dari label kelas menjadi nilai biner untuk setiap pengklasifikasi (baik kelas i atau bukan kelas i). Terakhir, kami menggunakan API pengoptimalan SciPy yang lebih baru untuk meminimalkan fungsi biaya untuk setiap pengklasifikasi. API mengambil fungsi objektif, kumpulan parameter awal, metode pengoptimalan, dan fungsi jacobian (gradien) jika ditentukan. Parameter yang ditemukan oleh rutin optimasi kemudian ditugaskan ke array parameter.

Salah satu bagian yang lebih menantang dalam mengimplementasikan kode vektorisasi adalah membuat semua interaksi matriks ditulis dengan benar, jadi saya merasa berguna untuk melakukan beberapa pemeriksaan kewarasan dengan melihat bentuk array/matriks yang saya kerjakan dan meyakinkan diri sendiri bahwa mereka masuk akal. Mari kita lihat beberapa struktur data yang digunakan dalam fungsi di atas.

In [10]:
rows = data['X'].shape[0]
params = data['X'].shape[1]

all_theta = np.zeros((10, params + 1))

X = np.insert(data['X'], 0, values=np.ones(rows), axis=1)

theta = np.zeros(params + 1)

y_0 = np.array([1 if label == 0 else 0 for label in data['y']])
y_0 = np.reshape(y_0, (rows, 1))

X.shape, y_0.shape, theta.shape, all_theta.shape

((5000, 401), (5000, 1), (401,), (10, 401))

These all appear to make sense.  Note that theta is a one-dimensional array, so when it gets converted to a matrix in the code that computes the gradient, it turns into a (1 X 401) matrix.  Let's also check the class labels in y to make sure they look like what we're expecting.

**Artinya**

Ini semua tampaknya masuk akal. Perhatikan bahwa theta adalah larik satu dimensi, jadi ketika diubah menjadi matriks dalam kode yang menghitung gradien, teta berubah menjadi matriks (1 X 401). Mari kita periksa juga label kelas di y untuk memastikannya terlihat seperti yang kita harapkan.

In [11]:
np.unique(data['y'])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

Let's make sure that our training function actually runs, and we get some sensible outputs, before going any further.

**Artinya**

Mari pastikan bahwa fungsi pelatihan kita benar-benar berjalan, dan kita mendapatkan hasil yang masuk akal, sebelum melangkah lebih jauh.

In [12]:
all_theta = one_vs_all(data['X'], data['y'], 10, 1)
all_theta

array([[-3.70247931e-05,  0.00000000e+00,  0.00000000e+00, ...,
        -2.24803602e-10,  2.31962907e-11,  0.00000000e+00],
       [-8.96250716e-05,  0.00000000e+00,  0.00000000e+00, ...,
         7.26120810e-09, -6.19965284e-10,  0.00000000e+00],
       [-8.39553387e-05,  0.00000000e+00,  0.00000000e+00, ...,
        -7.61695633e-10,  4.64917656e-11,  0.00000000e+00],
       ...,
       [-7.00832439e-05,  0.00000000e+00,  0.00000000e+00, ...,
        -6.92009029e-10,  4.29241494e-11,  0.00000000e+00],
       [-7.65187941e-05,  0.00000000e+00,  0.00000000e+00, ...,
        -8.09503274e-10,  5.31058721e-11,  0.00000000e+00],
       [-6.63412747e-05,  0.00000000e+00,  0.00000000e+00, ...,
        -3.49766172e-09,  1.13668635e-10,  0.00000000e+00]])

We're now ready for the final step - using the trained classifiers to predict a label for each image.  For this step we're going to compute the class probability for each class, for each training instance (using vectorized code of course!) and assign the output class label as the class with the highest probability.

**Artinya**

Kami sekarang siap untuk langkah terakhir - menggunakan pengklasifikasi terlatih untuk memprediksi label untuk setiap gambar. Untuk langkah ini kita akan menghitung probabilitas kelas untuk setiap kelas, untuk setiap instance pelatihan (tentu saja menggunakan kode vektor!) dan menetapkan label kelas keluaran sebagai kelas dengan probabilitas tertinggi.

In [13]:
def predict_all(X, all_theta):
    rows = X.shape[0]
    params = X.shape[1]
    num_labels = all_theta.shape[0]
    
    # same as before, insert ones to match the shape
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    
    # convert to matrices
    X = np.matrix(X)
    all_theta = np.matrix(all_theta)
    
    # compute the class probability for each class on each training instance
    h = sigmoid(X * all_theta.T)
    
    # create array of the index with the maximum probability
    h_argmax = np.argmax(h, axis=1)
    
    # because our array was zero-indexed we need to add one for the true label prediction
    h_argmax = h_argmax + 1
    
    return h_argmax

Now we can use the predict_all function to generate class predictions for each instance and see how well our classifier works.

**Artinya**

Sekarang kita dapat menggunakan fungsi predict_all untuk menghasilkan prediksi kelas untuk setiap instance dan melihat seberapa baik classifier kita bekerja.

In [14]:
y_pred = predict_all(data['X'], all_theta)
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, data['y'])]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print('accuracy = {0}%'.format(accuracy * 100))

accuracy = 74.6%


# **Kesimpulan**

Dalam melakukan prediksi Multi-Class Classification hal yang harus dilakukan adalah:
1. import library
2. import data
3. pembacaan dan penampilan data
4. membuat fungsi sigmoid
5. sekarang kita harus menulis fungsi cost untuk mengevaluasi solusi.
6. menghitung gradiet
7. membuat classifier 
  - k X (n + 1) array for the parameters of each of the k classifiers
  - insert a column of ones at the beginning for the intercept term
  - labels are 1-indexed instead of 0-indexed
  - minimize the objective function
8. theta adalah larik satu dimensi diubah menjadi matriks dalam kode yang menghitung gradien.
9. melakuakan cheking fungsi
10.  predict a label for each image
  - same as before, insert ones to match the shape
  - convert to matrices
  - compute the class probability for each class on each training instance
  - create array of the index with the maximum probability
  - because our array was zero-indexed we need to add one for the true label prediction
11. gunakan fungsi predict_all untuk menghasilkan prediksi kelas untuk setiap instance dan melihat seberapa baik classifier kita bekerja.